In [4]:
import kagglehub
import pandas as pd
import os

C:\Users\Hamza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
path = kagglehub.dataset_download("jakewright/200k-youtube-channel-analytics")
print("Data set path:", path)

Data set path: C:\Users\Hamza\.cache\kagglehub\datasets\jakewright\200k-youtube-channel-analytics\versions\1


In [6]:
dataset_files = os.listdir(path)
print("dataset Files", dataset_files)

dataset Files ['all_youtube_analytics.csv']


In [7]:
dataset_path = os.path.join(path, "all_youtube_analytics.csv")
data = pd.read_csv(dataset_path)
print("\nFirst few rows:")
print(data.head(2))
print("\nDataset Columns:", data.columns)


First few rows:
      video_id         day  views  redViews  comments  likes  dislikes  \
0  YuQaT52VEwo  2019-09-06    8.0       0.0       0.0    1.0       0.0   
1  YuQaT52VEwo  2019-09-07    7.0       0.0       0.0    0.0       0.0   

   videosAddedToPlaylists  videosRemovedFromPlaylists  shares  ...  \
0                     0.0                         0.0     0.0  ...   
1                     1.0                         0.0     0.0  ...   

   annotationClicks  annotationCloses  cardClickRate  cardTeaserClickRate  \
0               0.0               0.0            0.0                  0.0   
1               0.0               0.0            0.0                  0.0   

   cardImpressions  cardTeaserImpressions  cardClicks  cardTeaserClicks  \
0              0.0                    0.0         0.0               0.0   
1              0.0                    0.0         0.0               0.0   

   subscribersGained  subscribersLost  
0                0.0              0.0  
1          

In [8]:
if 'subscribersGained' in data.columns:
    print("\nDistribution of 'subscribersGained' column:")
    print(data['subscribersGained'].describe())
else:
    print("\n'subscribersGained' column not found.")
if 'subscribersLost' in data.columns:
    print("\nDistribution of 'subscribersLost' column:")
    print(data['subscribersLost'].describe())
else:
    print("\n'subscribersLost' column not found.")



Distribution of 'subscribersGained' column:
count    234889.000000
mean          0.167173
std           0.826207
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          31.000000
Name: subscribersGained, dtype: float64

Distribution of 'subscribersLost' column:
count    234889.000000
mean          0.004743
std           0.079772
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           9.000000
Name: subscribersLost, dtype: float64


In [9]:
print("\nMissing values in each column:")
print(data.isnull().sum())


Missing values in each column:
video_id                          0
day                               0
views                             0
redViews                          0
comments                          0
likes                             0
dislikes                          0
videosAddedToPlaylists            0
videosRemovedFromPlaylists        0
shares                            0
estimatedMinutesWatched           0
estimatedRedMinutesWatched        0
averageViewDuration               0
averageViewPercentage             0
annotationClickThroughRate        0
annotationCloseRate               0
annotationImpressions             0
annotationClickableImpressions    0
annotationClosableImpressions     0
annotationClicks                  0
annotationCloses                  0
cardClickRate                     0
cardTeaserClickRate               0
cardImpressions                   0
cardTeaserImpressions             0
cardClicks                        0
cardTeaserClicks                

In [10]:
print("\nSummary Statistics:")
print(data.describe())


Summary Statistics:
               views       redViews       comments          likes  \
count  234889.000000  234889.000000  234889.000000  234889.000000   
mean       88.842121      18.717326       0.039419       0.969816   
std       331.280375      78.293400       0.355816       3.984013   
min         0.000000       0.000000       0.000000     -11.000000   
25%         1.000000       0.000000       0.000000       0.000000   
50%         8.000000       2.000000       0.000000       0.000000   
75%        35.000000       8.000000       0.000000       0.000000   
max      8818.000000    2658.000000      24.000000     206.000000   

            dislikes  videosAddedToPlaylists  videosRemovedFromPlaylists  \
count  234889.000000           234889.000000               234889.000000   
mean        0.032215                1.262835                    0.228908   
std         0.241694                7.535244                    5.702640   
min       -19.000000                0.000000         

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [12]:
for column in data.columns:
    if data[column].dtype == 'object':
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:
        data[column].fillna(data[column].mean(), inplace=True)


C:\Users\Hamza\AppData\Local\Temp\ipykernel_5216\2349746636.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mode()[0], inplace=True)
C:\Users\Hamza\AppData\Local\Temp\ipykernel_5216\2349746636.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [13]:
label_encoders = {}
for column in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

In [14]:
X = data.drop('subscribersGained', axis=1)  # Replace 'target' with your target column name
y = data['subscribersGained']

In [15]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [16]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train, y_train)

In [ ]:
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))